In [1]:
%cd C:\SaiVinay\SproutsAI\GitHub_\ir_srgan_otsr\src_ir_imgs
import torch
import torch.nn as nn
from RRDBNet_arch import RRDBNet

ir_edge_netG = RRDBNet(in_nc=1, out_nc=1, nf=64, nb=23, gc=32)

#load the otsr model
path = r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_with_edgeloss\trained_models_edgeloss5e\40138_G.pth'
ir_edge_netG.load_state_dict(torch.load(path), strict=True)

C:\SaiVinay\SproutsAI\GitHub_\ir_srgan_otsr\src_ir_imgs


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\SaiVinay\\GitHub\\ir_srgan_otsr\\src_ir_with_edgeloss\\trained_models_edgeloss5e\\40138_G.pth'